In [1]:
from ipynb.fs.full.data_extraction import read_file,init_spark
from pyspark.sql.functions import count,when,corr
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from matplotlib import pyplot as plt

DataFrame[match_id: string, home_team_id: int, home_team: string, home_team_goal: int, league_id: int, B365H: double, B365D: double, B365A: double, away_team_id: int, away_team: string, away_team_goal: int, winner: string, country: string]


In [2]:
spark = init_spark()

In [3]:
player = read_file("Player.csv")
player_attributes = read_file("Player_Attributes.csv")

In [4]:
player_grouped = player_attributes.groupBy("player_api_id").mean()
original_columns = player_attributes.columns

In [ ]:
final_players = player_grouped.join(player,on="player_api_id")\
                .drop("player_fifa_api_id","id","birthday","avg(id)","avg(player_fifa_api_id)"
                      ,"avg(player_api_id)")

In [ ]:
for col in original_columns:
    if col != "player_api_id":
        final_players = final_players.withColumnRenamed("avg("+col+")",col)

In [ ]:
final_players = final_players.na.drop()

In [ ]:
features = [x for x in final_players.columns if x not in ["player_name","player_api_id"]]
assembler = VectorAssembler(inputCols = features,outputCol="features")
final_data = assembler.transform(final_players)

In [ ]:
kmeans = KMeans(distanceMeasure="cosine").setK(4).setSeed(1)
model = kmeans.fit(final_data.select("features"))
predictions = model.transform(final_data)
# final_players.select([count(when(final_players[c].isNull(), c)).alias(c) for c in final_players.columns]).show()

In [ ]:
# correlation = {}
# for col in predictions.columns:
#     if col!="features" and col!="prediction":
#         c = predictions.select(corr(predictions[col],predictions["prediction"])).collect()[0][0]
#         correlation[col] = c

In [ ]:
predictions.filter(predictions["prediction"] == 0).select("player_name","prediction","overall_rating").show()